# ReverseNumber Example - two users

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_get_instance_worker_thread

def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

# Create two users

In [3]:
cls = colink_utils.start_colink_server_with_users(num_users=2)

# USER 0

In [4]:
run_dispatch_worker_thread(cls[0])
run_get_instance_worker_thread(cls[0])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  02acba5dea871c06c99ba42bb813eeef11074940671bd3a950f89ebed350da52a3

~~~ serving get_instances request ~~~
Mounted c9257714-8465-4951-8c15-e4aefaa58cc3 at flows:reverse_number_atomic:mounts:02d29bc3e276de86016c350d3e3b637148eda4af310c0f84405c0f138365aab669:c9257714-8465-4951-8c15-e4aefaa58cc3
Fetched singleton c9257714-8465-4951-8c15-e4aefaa58cc3

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_atomic
flow_id: c9257714-8465-4951-8c15-e4aefaa58cc3
message_paths: ['push_tasks:22b5208d-2d44-497e-a930-b0958b2ccc73:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_atomic
flow_id: c9257714-8465-4951-8c15-e4aefaa58cc3
message_paths: ['push_tasks:816601b3-86b0-4c12-9cb1-b2177c024ebd:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}


In [5]:
serve_utils.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True


True

# USER 1

In [6]:
run_dispatch_worker_thread(cls[1])
run_get_instance_worker_thread(cls[1])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  02d29bc3e276de86016c350d3e3b637148eda4af310c0f84405c0f138365aab669

~~~ get_instances initiator ~~~
Received subflow instances: {
    "first_reverse_flow": {
        "flow_id": "c9257714-8465-4951-8c15-e4aefaa58cc3",
        "successful": true,
        "message": "Fetched local flow instance."
    },
    "second_reverse_flow": {
        "flow_id": "c9257714-8465-4951-8c15-e4aefaa58cc3",
        "successful": true,
        "message": "Fetched local flow instance."
    }
}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 3180a6d9-7eb5-4ec3-8c7b-3c9c08f4fbdc
message_paths: ['push_tasks:19a6afcb-770b-4dac-af51-7946f01e12fe:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 3180a6d9-7eb5-4ec3-8c7b-3c9c08f

In [7]:
serve_utils.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential",
)

Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

In [8]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)

Mounted 3180a6d9-7eb5-4ec3-8c7b-3c9c08f4fbdc at flows:reverse_number_sequential:mounts:local:3180a6d9-7eb5-4ec3-8c7b-3c9c08f4fbdc


In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0.

In [10]:
status(cls[0])

 reverse_number_atomic/
   flow_class_name
   singleton
   parallel_dispatch
   mounts/
     02d29bc3e276de86016c350d3e3b637148eda4af310c0f84405c0f138365aab669/
       c9257714-8465-4951-8c15-e4aefaa58cc3/
         config_overrides
         state
         init
   init
   default_dispatch_point

All flow instances:
 c9257714-8465-4951-8c15-e4aefaa58cc3


In [11]:
status(cls[1])

 reverse_number_sequential/
   default_dispatch_point
   init
   flow_class_name
   parallel_dispatch
   singleton
   mounts/
     local/
       3180a6d9-7eb5-4ec3-8c7b-3c9c08f4fbdc/
         state
         init
         config_overrides

All flow instances:
 3180a6d9-7eb5-4ec3-8c7b-3c9c08f4fbdc
